In [1]:
!pip install opencv-python

In [2]:
import tensorflow as tf # this is to import tensorflow
import json # this is to import json
import pandas as pd # this is to import pandas
import numpy as np # this is to import numpy
from skimage import io, transform # this is to import skimage
from sklearn.model_selection import train_test_split # this is just for train test split lol
import cv2 # compooter vision
from tensorflow.keras import datasets, layers, models, losses, Model # this imports things
from tqdm.notebook import tqdm # this is literally just a progress bar
import os # operating system things
import glob # this is because file pathnames are a big aaaaaaaaaauuuuuuuuugh

In [53]:
# Read the json
with open("amogus.json", 'r') as f:
    data_df = pd.json_normalize([i for i in json.loads(f.read()) if i is not None])
data_df = data_df.set_index("id")
data_df

,url_256,url_1024,coordinates,country
id,,,,
1509662729539070,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]",null
1587956288269248,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]",null
1414341782427856,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]",null
1504003806771142,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[-9.5395717634566e-07, 5.4042628244286e-08]",null
1502346620201016,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]",null
...,...,...,...,...
4204140206347239,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[99.206494380421, 9.4558343148341]",Thailand
1405412416480572,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[99.169389331325, 9.3891869120472]",Thailand
3963740523712531,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[99.114069704404, 9.6701075610815]",Thailand


In [54]:
data_df.groupby(['country']).count()

,url_256,url_1024,coordinates
country,,,
Afghanistan,81,81,81
Albania,39,39,39
Algeria,491,491,491
Andorra,4,4,4
Angola,238,238,238
...,...,...,...
Yemen,66,66,66
Zambia,462,462,462
Zimbabwe,127,127,127


In [5]:
# images = {}
# folder = "img2"
# for filename in tqdm(os.listdir(folder)):
#     img = cv2.imread(os.path.join(folder,filename))
#     if img is not None:
#         if img.shape!=(128,256,3):
#             img = (255 * transform.resize(img, (128,256))).astype(np.uint8)
#         images[os.path.splitext(filename)[0]] = img
        
# reshape image
def reshape(img):
    if img.shape!=(192,256,3):
        img = (255 * transform.resize(img, (192,256))).astype(np.uint8)
    return img

In [6]:
images = {file: cv2.imread(file) for file in tqdm(glob.glob("img2/*.jpg"))}

  0%|          | 0/76440 [00:00<?, ?it/s]

In [47]:
img_series = pd.Series(images)
img_series = img_series[img_series.notna()].copy()
img_series = img_series[[i.shape==(192, 256, 3) for i in img_series]].copy() # change to reshape
# img_series = img_series.apply(lambda x: reshape(x))
img_series = img_series.rename(lambda str: str[5:-4]) # convert file name e.g. "img2\1000148914149750.jpg"->1000148914149750
img_series
# img_series.apply(lambda x:x.shape).value_counts() # see different aspect ratios and their freqs

(144, 256, 3)    32113
(192, 256, 3)    31661
(128, 256, 3)     4892
(170, 255, 3)      615
(153, 255, 3)      582
                 ...  
(159, 254, 3)        1
(88, 255, 3)         1
(117, 255, 3)        1
(208, 255, 3)        1
(61, 253, 3)         1
Length: 282, dtype: int64

In [22]:
data_df["img"] = img_series
deez = data_df.dropna()
deez

,url_256,url_1024,coordinates,country,img
id,,,,,
1509662729539070,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]",null,"[[[144, 150, 157], [144, 150, 157], [144, 150,..."
1414341782427856,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]",null,"[[[124, 127, 101], [255, 255, 237], [182, 181,..."
1504003806771142,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[-9.5395717634566e-07, 5.4042628244286e-08]",null,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1502346620201016,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]",null,"[[[222, 231, 234], [214, 223, 226], [215, 224,..."
1548739535521112,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[3.1402148603315e-09, -2.7646415881002e-09]",null,"[[[98, 103, 104], [98, 103, 104], [97, 102, 10..."
...,...,...,...,...,...
3894961763872349,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[99.130769289577, 9.5050045863067]",null,"[[[114, 120, 119], [61, 67, 66], [11, 17, 16],..."
4290124437684494,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[99.131618903251, 9.5203148594143]",null,"[[[156, 154, 154], [164, 162, 162], [157, 157,..."
1405412416480572,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[99.169389331325, 9.3891869120472]",null,"[[[151, 147, 146], [151, 147, 146], [151, 147,..."


In [15]:
# deez.groupby(['country']).count()

,url_256,url_1024,coordinates,img
country,,,,
Brazil,2202,2202,2202,2202
Colombia,1027,1027,1027,1027
Congo,1010,1010,1010,1010
Democratic Republic of Congo,2527,2527,2527,2527
Ecuador,770,770,770,770
Gabon,1057,1057,1057,1057
Indonesia,1557,1557,1557,1557
Kazakhstan,1,1,1,1
Kenya,1389,1389,1389,1389


In [24]:
from sklearn.preprocessing import OneHotEncoder

country_df = deez[deez["country"] != "null"]

encoder = OneHotEncoder(handle_unknown='ignore')

enc_country_df = pd.DataFrame(encoder.fit_transform(country_df[['country']]).toarray())
enc_country_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
# img_series = img_series[[i.shape==(192, 256, 3) for i in img_series]].copy()
# img_series

In [11]:
# ids = [os.path.splitext(os.path.basename(filename))[0] for filename in tqdm(img_series.index)]
# imgs = img_series.values

In [25]:
encoder.categories_

[array(['Brazil', 'Colombia', 'Congo', 'Democratic Republic of Congo',
        'Ecuador', 'Gabon', 'Indonesia', 'Kazakhstan', 'Kenya',
        'Sao Tome and Principe', 'Somalia', 'Uganda'], dtype=object)]

In [12]:
# coords = np.stack(data_df.loc[ids, "coordinates"])
# coords

In [46]:
country_df.img.apply(lambda x:x.shape).value_counts()

(192, 256, 3)    12842
Name: img, dtype: int64

In [40]:
country_df

,url_256,url_1024,coordinates,country,img
id,,,,,
251658540025862,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[-49.9833142, -0.1628493]",Brazil,"[[[130, 112, 95], [132, 114, 97], [134, 116, 9..."
3225189954378529,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[-49.983114997753, -0.16297426741573]",Brazil,"[[[82, 72, 62], [82, 72, 62], [82, 72, 62], [8..."
3961817710599361,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[-76.467011264588, -0.60972357513499]",Ecuador,"[[[198, 198, 168], [180, 178, 148], [187, 182,..."
1464964297198862,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[-76.15782550331, -0.075347415318279]",Ecuador,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1089979744858814,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[-76.424191356872, -0.69611050477275]",Ecuador,"[[[68, 112, 95], [74, 118, 101], [77, 119, 101..."
...,...,...,...,...,...
1392914834376447,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[38.749054129822, 9.0557014480259]",Kenya,"[[[180, 117, 73], [180, 117, 73], [179, 116, 7..."
1409282039405690,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[40.210672635245, 9.0306870237748]",Kenya,"[[[228, 209, 224], [241, 222, 237], [255, 239,..."
820611748876368,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[40.417454200002, 9.0645206000024]",Kenya,"[[[191, 170, 168], [191, 170, 168], [190, 169,..."


In [26]:
X_train_index, X_test_index, y_train, y_test = train_test_split(list(country_df.index), enc_country_df, test_size=0.20)

(['1253434511744354',
  '1401513063583382',
  '1042632349781217',
  '142679274486948',
  '830576404477872',
  '1322367384826228',
  '1293893404414054',
  '121371323316453',
  '564125227887568',
  '1365973247122511',
  '1439371929731007',
  '2219474681522156',
  '3961937787177381',
  '1238488626569580',
  '483246009466565',
  '501311360893026',
  '1363370900699897',
  '510833193254852',
  '970210243741472',
  '1227231871043546',
  '1617316581992189',
  '948349135930407',
  '3139714292925023',
  '2865385117124481',
  '1790762311103166',
  '949230795618489',
  '1353122081736294',
  '1646755329047653',
  '2871250919859467',
  '756496691731853',
  '1373217696398876',
  '2924363137778913',
  '1239290530152888',
  '2186295184864198',
  '324678889552516',
  '479303183487274',
  '1222573541493160',
  '1477632999250413',
  '2078217242320703',
  '4280136642095324',
  '1228439904285936',
  '169771011694314',
  '944039840372526',
  '671270910843762',
  '292365059206057',
  '1511388802587739',
  '28

In [27]:
X_train = np.stack(img_series[X_train_index].values)
X_test = np.stack(img_series[X_test_index].values)

In [28]:
resnet_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (192,256,3))
for layer in resnet_model.layers:
    layer.trainable = False

234698864/234698864 [==============================] - 117s 0us/step


In [37]:
model = models.Sequential()
#model.add(layers.Lambda(lambda image: tf.image.resize(image, (128, 256))))
model.add(resnet_model)
model.add(layers.Flatten())
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(len(enc_country_df.columns), activation="linear"))

In [38]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [39]:
model.fit(X_train[:1000], y_train[:1000], batch_size=64, epochs=5, validation_data=(X_test[:100], y_test[:100]))

Epoch 1/5
16/16 [==============================] - 3015s 185s/step - loss: 1.9174 - accuracy: 0.0970 - val_loss: 8.3930 - val_accuracy: 0.1800
Epoch 2/5
16/16 [==============================] - 1609s 96s/step - loss: 1.2853 - accuracy: 0.1610 - val_loss: 1.4899 - val_accuracy: 0.1800
Epoch 3/5
16/16 [==============================] - 1034s 65s/step - loss: 0.9793 - accuracy: 0.1910 - val_loss: 0.5672 - val_accuracy: 0.2500
Epoch 4/5
16/16 [==============================] - 931s 59s/step - loss: 0.7380 - accuracy: 0.2160 - val_loss: 0.3378 - val_accuracy: 0.2600
Epoch 5/5
16/16 [==============================] - 1036s 65s/step - loss: 0.6051 - accuracy: 0.2400 - val_loss: 0.2183 - val_accuracy: 0.2900


In [ ]:
from collections import Counter
Counter([i.shape for i in tqdm(img_series)])

In [ ]:
theimg = reshape(io.imread("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS2RBxzDiJKUypDZkMceKYNj9jrix5r2OcM5Q&usqp=CAU"))

In [ ]:
model.predict(X_test[380][None, ...])

In [ ]:
io.imshow(X_test[380])

In [ ]:
y_test[380]